In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import enum
import re
import nltk 
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [2]:
#Veri setinin yüklenmesi ve örnek veri
dataset = pd.read_csv("data.csv",delimiter=";",header=None,names=["Review","Rating"])
print("Verisetinde {} adet cümle mevcut.".format(len(dataset)))
dataset.head()

Verisetinde 50000 adet cümle mevcut.


,Review,Rating
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [3]:
#Veri ön işleme
#Ön işleme öncesi örnek cümle
print(dataset['Review'].values[0],"\n\n")

from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

#Stemmer nesnesi oluşturulması
porter = PorterStemmer() 
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

#büyük harflerin küçük harfe çevrilmesi
dataset['Review'] = dataset['Review'].apply(lambda x: x.lower())

# Özel karakterlerin(noktalama işareti vs) çıkartılması
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"\W", " ", x))

# tek karakterlerin boşluk ile değiştirilmesi
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"\s+[a-zA-Z]\s+", " ", x))

# en baştan tek kalan karakterlerin çıkartılması
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"\^[a-zA-Z]\s+", " ", x))

# Birden fazla boşluğun tek boşlukla değiştirilmesi
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"\s+", " ", x))

# b öneklerinin silinmesi
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"^b\s+", " ", x))

#fazladan boşlukların temizlenmesi
dataset['Review'] = dataset['Review'].apply(lambda x: x.strip())

#stemming işlemi
dataset['Review'] = dataset['Review'].apply(lambda x: stemSentence(x))

data = dataset['Review'].values.tolist()
target = dataset['Rating'].values.tolist()

#Ön işleme sonrası aynı cümle
print(data[0])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [4]:
#Cümlelerin eğitim ve test olarak ayrılması %90 Eğitim %10 test
ratio = int(len(data) * .90)
x_train, y_train = data[:ratio], target[:ratio]
y_train = np.array(y_train)
x_test, y_test   = data[ratio:], target[ratio:]
y_test = np.array(y_test)

print("{} adet cümle eğitim için kullanılacak.".format(len(x_train)))
print("{} adet cümle test için kullanılacak.".format(len(x_test)))

45000 adet cümle eğitim için kullanılacak.
5000 adet cümle test için kullanılacak.


In [5]:
#Cümlelerin içinde geçen kelimelerden 10000 kelimelik bir sözlük oluşturuluyor.
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)
#tokenizer.word_index

#Cümleler sayılara dönüştürülüyor
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)

#Cümlelerin önceki ve sonraki hallerinin görüntülenmesi
IDX = 0
print("Öncesi: {}".format(x_train[IDX]))
print("Sonrası: {}".format(np.array(x_train_tokens[IDX])))

Öncesi: one of the other review ha mention that after watch just 1 oz episod you ll be hook they are right as thi is exactli what happen with me br br the first thing that struck me about oz wa it brutal and unflinch scene of violenc which set in right from the word go trust me thi is not show for the faint heart or timid thi show pull no punch with regard to drug sex or violenc it is hardcor in the classic use of the word br br it is call oz as that is the nicknam given to the oswald maximum secur state penitentari it focus mainli on emerald citi an experiment section of the prison where all the cell have glass front and face inward so privaci is not high on the agenda em citi is home to mani aryan muslim gangsta latino christian italian irish and more so scuffl death stare dodgi deal and shadi agreement are never far away br br would say the main appeal of the show is due to the fact that it goe where other show wouldn dare forget pretti pictur paint for mainstream audienc forget cha

In [6]:
#RNN'e girdileri vermeden önce tamamının aynı boyutta olması gerekli. Bu sebeple aşağıdaki matematiksel işlemleri yapıyoruz.
total_sentences = x_train_tokens + x_test_tokens
num_tokens = np.array([len(tokens) for tokens in total_sentences])
#print(np.mean(num_tokens))
#print(np.std(num_tokens))
#print(np.max(num_tokens))
#print(np.min(num_tokens))

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens) # np.std = standart sapma
max_tokens = int(max_tokens)
print(max_tokens)
#Verinin ne kadarını bu kapsama aldığımızın ölçülmesi
print("%", round(np.sum(num_tokens < max_tokens) / len(num_tokens) * 100, 2))

545
% 94.54


In [7]:
#Padding işlemi. Bulunan uzunluk değerine göre cümlelerin yeniden düzenlenmesi. Kısa olanların başına sıfır eklenmesi.
#Uzun olanlardan baştan silme yapılması
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad  = pad_sequences(x_test_tokens,  maxlen=max_tokens)

In [8]:
#RNN oluşturma
#ardışık bir model
model = Sequential()
    
#her kelimeye karşılık gelen 50 uzunluğunda bir vektör oluşturulur. (Embedding matrisi)
embedding_size = 50
    
#matris kelime sayısı ve embedding büyüklüğünde olacak, yani 10bine 50 uzunluğunda 
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))
#LSTM layerlerinin eklenmesi
# 16 nöronlu LSTM (16 outputlu , return_sequences=True demek output'un tamamını ver demek)
model.add(GRU(units=16, return_sequences=True))
# 8 nöronlu LSTM (8 outputlu , return_sequences=True demek output'un tamamını ver demek)
model.add(GRU(units=8, return_sequences=True))
# 4 nöronlu LSTM (4 outputlu , return_sequences=False yani default değer, tek bir output verecek)
model.add(GRU(units=4))
# Tek bir nörondan oluşan output layer'ı
model.add(Dense(1, activation='sigmoid'))

#modelin derlenmesi 
#iki sınıf olduğu için loss fonksiyonu olarak binary_crossentropy 
#modelin başarısını görmek için accuracy metrics
#optimizasyon algoritması
optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [9]:
#Modelin özeti
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 545, 50)           500000    
_________________________________________________________________
gru (GRU)                    (None, 545, 16)           3264      
_________________________________________________________________
gru_1 (GRU)                  (None, 545, 8)            624       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 168       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 504,061
Trainable params: 504,061
Non-trainable params: 0
_________________________________________________________________


In [10]:
#model eğitimi, bir defa eğitimden geçmesi -> epoch , batch_size -> 256'şar 256'şar beslenecek.
model.fit(x_train_pad, y_train, epochs=5, batch_size=256)

Epoch 1/5
176/176 [==============================] - 85s 451ms/step - loss: 0.6133 - accuracy: 0.6389
Epoch 2/5
176/176 [==============================] - 81s 461ms/step - loss: 0.3074 - accuracy: 0.8799
Epoch 3/5
176/176 [==============================] - 81s 460ms/step - loss: 0.2189 - accuracy: 0.9209
Epoch 4/5
176/176 [==============================] - 81s 459ms/step - loss: 0.1816 - accuracy: 0.9381
Epoch 5/5
176/176 [==============================] - 81s 460ms/step - loss: 0.1534 - accuracy: 0.9516


In [11]:
#Evaluate fonksiyonu yalnızca accuracy ve loss değerini döndürür
result = model.evaluate(x_test_pad, y_test)

num_true_sentence = int(len(x_test) * result[1])
print("Test verisindeki {} adet cümleden {} tanesi doğru bilindi.".format(len(x_test), num_true_sentence))

157/157 [==============================] - 9s 51ms/step - loss: 0.2900 - accuracy: 0.8966
Test verisindeki 5000 adet cümleden 4483 tanesi doğru bilindi.


In [12]:
#tek tek cümlelerin sonuçlarını görmek için predict metodu kullanılması
y_pred = model.predict(x_test_pad)

#Her cümle için çıktı 0 ile 1 arasındadır. 0 olumsuz 1 olumlu anlamındadır. 
#0.5 üzerini olumlu altını olumsuz olarak işaretleyelim.
y_pred = np.array([1 if p>0.5 else 0 for p in y_pred])

#Bir örnek üzerinde inceleyelim.
IDX = 0
sentence = x_test[IDX]
real_rate = y_test[IDX]
predicted_rate = y_pred[IDX]

print("Cümle: {} \nAsıl Etiket: {} \nÜretilen Etiket: {}".format(sentence, real_rate, predicted_rate))

Cümle: what enjoy most in thi film wa the sceneri of corfu be greek ador my countri and like the flatter director point of view base on true stori dure the year when greec wa struggl to stand on her own two feet through war nazi and hardship an italian soldier and greek girl fall in love but the time are hard and they have lot of sacrific to make nichola cage look great in uniform give passion account of thi unfulfil in the begin love ador christian bale play mandra the heroin husband to be he look veri veri good as greek hi person match the one of the greek patriot true fighter in there or what one of the movi would like to buy and keep it in my collect for ever  
Asıl Etiket: 1 
Üretilen Etiket: 1


In [15]:
#Accuracy değeri hariç precision,recall ve f-measure değerlerine bakalım.
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[2271  259]
 [ 258 2212]]
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      2530
           1       0.90      0.90      0.90      2470

    accuracy                           0.90      5000
   macro avg       0.90      0.90      0.90      5000
weighted avg       0.90      0.90      0.90      5000

